# IMPORTS

## Libraries

In [1]:
import pandas as pd
import numpy as np
#import random
import datetime
#import pickle
import json
import requests

# import warnings
# warnings.filterwarnings("ignore")

# import seaborn as sns
# import matplotlib.pyplot as plt
# from IPython.core.display import HTML

# from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_absolute_error, mean_squared_error
# from sklearn.linear_model import LinearRegression, Lasso

# import xgboost as xgb
# from lightgbm import LGBMRegressor

# API Test >> HEROKU

In [2]:
#salesRaw = pd.read_csv('../../01-Data/train.csv', low_memory=False)
storeRaw = pd.read_csv('../../01-Data/store.csv', low_memory=False)
testRaw = pd.read_csv('../../01-Data/test.csv', low_memory=False)

In [32]:
storeList = [30, 14, 23]

# Merge Test dataset + Store
dfTest = pd.merge(testRaw, storeRaw, how='left', on='Store')

# Choose Store for Prediction
dfTest = dfTest[dfTest['Store'].isin(storeList)]

# Remove Closed Days
dfTest = dfTest[dfTest['Open'] != 0]
dfTest = dfTest[~dfTest['Open'].isnull()]
dfTest = dfTest.drop('Id', axis=1)

In [33]:
# Convert DataFrame to JSON
data = json.dumps(dfTest.to_dict(orient='records'))

In [34]:
## API Call
# url = 'http://127.0.0.1:5000/rossmann/predict'
url = 'https://model-rossmann.herokuapp.com/rossmann/predict'
header = {'Content-Type': 'application/json'}
data = data

r = requests.post(url, data=data, headers=header)
print('Status Code {}'.format(r.status_code))

Status Code 200


In [35]:
dfResponse = pd.DataFrame(r.json(), columns=r.json()[0].keys())

In [36]:
dfResponse.head()

,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,...,Year,Month,Day,WeekOfYear,YearWeek,CompetionSinse,CompetionTimeMonth,PromoSince,PromoTimeWeek,Prediction
0,14,4,2015-09-17T00:00:00.000Z,1.0,1,Regular Day,0,a,basic,1300.0,...,2015,9,17,38,2015-37,2014-03-01T00:00:00.000Z,18,2011-09-26T00:00:00.000Z,207,6386.439453
1,23,4,2015-09-17T00:00:00.000Z,1.0,1,Regular Day,0,d,basic,4060.0,...,2015,9,17,38,2015-37,2005-08-01T00:00:00.000Z,123,2015-09-14T00:00:00.000Z,0,7098.622559
2,30,4,2015-09-17T00:00:00.000Z,1.0,1,Regular Day,0,a,basic,40.0,...,2015,9,17,38,2015-37,2014-02-01T00:00:00.000Z,19,2014-03-03T00:00:00.000Z,80,5164.730469
3,14,3,2015-09-16T00:00:00.000Z,1.0,1,Regular Day,0,a,basic,1300.0,...,2015,9,16,38,2015-37,2014-03-01T00:00:00.000Z,18,2011-09-26T00:00:00.000Z,207,6444.190918
4,23,3,2015-09-16T00:00:00.000Z,1.0,1,Regular Day,0,d,basic,4060.0,...,2015,9,16,38,2015-37,2005-08-01T00:00:00.000Z,123,2015-09-14T00:00:00.000Z,0,7195.125977


In [37]:
dfResponse2 = dfResponse[['Store', 'Prediction']].groupby('Store').sum().reset_index()
dfResponse2

,Store,Prediction
0,14,240417.760742
1,23,274111.687500
2,30,203692.620117


In [38]:
dfResponse2 = dfResponse[['Store', 'Prediction']].groupby('Store').sum().reset_index()

for i in range(len(dfResponse2)):
    print('Store Number {} will sell R${:,.2f} in the next 6 weeks'.format(
            dfResponse2.loc[i, 'Store'],
             dfResponse2.loc[i, 'Prediction']))

Store Number 14 will sell R$240,417.76 in the next 6 weeks
Store Number 23 will sell R$274,111.69 in the next 6 weeks
Store Number 30 will sell R$203,692.62 in the next 6 weeks


In [30]:
data

'[{"Store": 14, "DayOfWeek": 4, "Date": "2015-09-17", "Open": 1.0, "Promo": 1, "StateHoliday": "0", "SchoolHoliday": 0, "StoreType": "a", "Assortment": "a", "CompetitionDistance": 1300.0, "CompetitionOpenSinceMonth": 3.0, "CompetitionOpenSinceYear": 2014.0, "Promo2": 1, "Promo2SinceWeek": 40.0, "Promo2SinceYear": 2011.0, "PromoInterval": "Jan,Apr,Jul,Oct"}, {"Store": 23, "DayOfWeek": 4, "Date": "2015-09-17", "Open": 1.0, "Promo": 1, "StateHoliday": "0", "SchoolHoliday": 0, "StoreType": "d", "Assortment": "a", "CompetitionDistance": 4060.0, "CompetitionOpenSinceMonth": 8.0, "CompetitionOpenSinceYear": 2005.0, "Promo2": 0, "Promo2SinceWeek": NaN, "Promo2SinceYear": NaN, "PromoInterval": NaN}, {"Store": 30, "DayOfWeek": 4, "Date": "2015-09-17", "Open": 1.0, "Promo": 1, "StateHoliday": "0", "SchoolHoliday": 0, "StoreType": "a", "Assortment": "a", "CompetitionDistance": 40.0, "CompetitionOpenSinceMonth": 2.0, "CompetitionOpenSinceYear": 2014.0, "Promo2": 1, "Promo2SinceWeek": 10.0, "Promo2S